In [1]:
# 1) find the target kernel name and
# 2) read gpu trace file
# 3) read gpu metrics
# 4) rearrange the information and combine them in the same dataframe
# 5) save the result as csv file

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep

In [3]:
bs_size = [16, 32, 64, 128, 256, 512, 1024]

target_kern_name = ['spPostprocess2D_kernel', 'spPreprocess2D_kernel', 'spProcess2D_kernel']

out_columns = None
df_kern = None
count = 0

for bs in bs_size:
    trace_file = 'trace'+str(bs)+'.csv'
    metrics_file = 'metrics'+str(bs)+'.csv'
    
    # read trace and metrics into dataframe, from generated trace file
    df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)
    
    if count == 0:
        out_columns = prep.init_df_columns(df_metrics)
        # empty df
        df_kern = pd.DataFrame(columns=out_columns)
    
    for mykern in target_kern_name:
        # current kernel trace
        df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics, mykern, out_columns)
        # add current trace
        df_kern = df_kern.append(df_current, ignore_index=True)

    count += 1


In [4]:
#df_kern

In [5]:
df_kern = prep.Groupbyname(df_kern, target_kern_name, out_columns)

In [6]:
df_kern

,kern_name,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,shared_mem,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,sppostprocess2d_kernel,0.369442,65536.0,1.0,1.0,16.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.579743,2.669206%,0.000000%
1,sppostprocess2d_kernel,0.369665,32768.0,1.0,1.0,32.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.046022,2.662936%,0.000000%
2,sppostprocess2d_kernel,0.370338,16384.0,1.0,1.0,64.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.144447,2.651583%,0.000000%
3,sppostprocess2d_kernel,0.371009,8192.0,1.0,1.0,128.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.180603,2.652395%,0.000000%
4,sppostprocess2d_kernel,0.369538,4096.0,1.0,1.0,256.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.384790,2.650207%,0.000000%
5,sppostprocess2d_kernel,0.369121,2048.0,1.0,1.0,512.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.868877,2.910863%,0.000000%
6,sppostprocess2d_kernel,0.369729,1024.0,1.0,1.0,1024.0,1.0,1.0,22.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.046537,2.657424%,0.000000%
7,sppreprocess2d_kernel,0.367554,65536.0,1.0,1.0,16.0,1.0,1.0,21.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.503311,1.892745%,0.000000%
8,sppreprocess2d_kernel,0.361954,32768.0,1.0,1.0,32.0,1.0,1.0,21.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.042429,1.899950%,0.000000%
9,sppreprocess2d_kernel,0.364290,16384.0,1.0,1.0,64.0,1.0,1.0,21.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.140287,1.900207%,0.000000%


In [7]:
df_kern.to_csv('results.csv', index=False, encoding='utf-8')